### Import packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import shutil
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import random
import cv2
import h5py
import imutils
import glob
import os
import urllib.request
from tqdm import tqdm
import pickle

from keras.models import Sequential, Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation
from keras.layers import UpSampling2D, BatchNormalization, Flatten, Dense, GlobalMaxPooling2D
from keras.layers.core import Dropout, Reshape
from keras.layers.merge import Concatenate
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.optimizers import RMSprop, Adam

seed = 0
np.random.seed(seed)

### Download and read data

In [5]:
train = pd.read_csv('./Data/Share/myntra_train_dataset.csv')

In [6]:
test = pd.read_csv('./Data/Share/Submission_offline3d61f7e.csv')

In [6]:
offline_test = pd.read_csv('./Data/Share/Submission_offline3d61f7e.csv')
faulty_test_offline = []
for url in tqdm(offline_test['Link_to_the_image'].dropna().drop_duplicates()):
    name = url[url.rfind('/')+1:]
    if os.path.exists('./Data/Test_offline/'+name):
        continue
    try:
        urllib.request.urlretrieve(url, './Data/Test_offline/'+name)
    except:
        print(url)
        faulty_test_offline.append(url)

  0%|          | 62/14752 [02:11<111:54:07, 27.42s/it]

http://repo.karvyecom.com/image-repository/Classic/PERLE-CTS/PERLE-CTS_1.jpg


  1%|▏         | 186/14752 [02:11<54:22:31, 13.44s/it]

http://cdn.myeshopbox.com/flipkart_images/duke-75-styles-18lot/SDVP10Q_Ambrosia/COMBO_1.jpg


  4%|▍         | 562/14752 [02:11<18:10:22,  4.61s/it]

...

 90%|█████████ | 13333/14752 [26:13<01:41, 13.91it/s]

http://cdn.myeshopbox.com/flipkart_images/Duke-36-mix-lot-images/BBAPLDK34350/FLIPKART_2.jpg
http://cdn.myeshopbox.com/flipkart_images/duke-45-styles-lot24/LF2778_Cranberry/FLIPKART_2.jpg


 98%|█████████▊| 14434/14752 [36:41<3:29:10, 39.47s/it]

http://repo.karvyecom.com/image-repository/Classic/C-4-SEASONVA-113/C-4-SEASONVA-113_1.jpg


100%|██████████| 14752/14752 [42:06<00:00,  1.14it/s]  


### Extract T-shirt

In [4]:
def boundingBox(edges, im):
    gray = edges.astype('uint8')
    im_h = gray.shape[0]
    im_w = gray.shape[1]
    cnts = cv2.findContours(gray,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]
     
    x_l, y_l, x_r, y_r = [], [], [], []
    
    # loop over the digit area candidates
    for c in cnts:
        # compute the bounding box of the contour
        (x, y, w, h) = cv2.boundingRect(c)
        
        x_l.append(x)
        y_l.append(y)
        x_r.append(x+w)
        y_r.append(y+h)
        
    
    # Getting the outer most bounding box
    x_l = min(x_l)
    y_l = min(y_l)
    x_r = max(x_r)
    y_r = max(y_r)
        
    return im[y_l:y_r, x_l:x_r, :]

In [5]:
def extractTshirt(im):
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3, 3), 0)
    v = np.median(gray)
    sigma = 0.33

    #---- apply optimal Canny edge detection using the computed median----
    lower_thresh = int(max(0, (1.0 - sigma) * v))
    upper_thresh = int(min(255, (1.0 + sigma) * v))
    edges = cv2.Canny(blur, lower_thresh, upper_thresh)
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.morphologyEx(edges, cv2.MORPH_DILATE, kernel)
    bounded = boundingBox(dilated, im)
    if bounded.shape[0]<1250:
        bounded = bounded
    else:
        height = bounded.shape[0]
        width = bounded.shape[1]
        bounded = bounded[int(1.3*height//4):int(3.5*(height//4)), :, :]
    return bounded

In [ ]:
dims_test = []
for im_path in tqdm(glob.glob('./Data/Test_offline/*')):
    im = cv2.imread(im_path)
    if im is not None:
        out_file = './Data/Test_offline_cleaned/'+im_path[im_path.rfind('/')+1:im_path.rfind('.')]+'.jpg'
        if not os.path.exists(out_file):
            im = extractTshirt(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
            cv2.imwrite(out_file, cv2.cvtColor(im, cv2.COLOR_RGB2BGR))
            dims_test.append(im.shape)

### Keras Model

In [7]:
labels = pd.get_dummies(train['Sub_category'])
labels.index = train.Link_to_the_image.astype(str).map(lambda x : x[x.rfind('/')+1:x.rfind('.')]+'.jpg')

In [8]:
test_files = glob.glob('./Data/Test_offline_cleaned/*')

In [9]:
width = 75
height = 75
channels = 3

#### Test

In [10]:
x_im_test = np.empty((len(test_files), height, width, channels))
link_test = []
for idx, im_path in enumerate(tqdm(test_files)):
    im = cv2.cvtColor(cv2.imread(im_path), cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (width, height), interpolation=cv2.INTER_AREA).reshape((height, width, channels))
    im_path = im_path[im_path.rfind('/')+1:]
    x_im_test[idx] = im
    link_test.append(im_path)

100%|██████████| 14701/14701 [03:39<00:00, 66.92it/s]


In [12]:
with h5py.File('./Data/test_data_offline.h5', 'w') as hf:
    hf.create_dataset("x_im",  data=x_im_test, maxshape=(None, height, width, channels))
pickle.dump(link_test, open('./Data/test_data_offline_links.pickle', 'wb'))

In [14]:
with h5py.File('./Data/test_data_offline.h5', 'r') as hf:
    x_im_test = hf['x_im'][:]/255.0
link_test = pickle.load(open('./Data/test_data_offline_links.pickle', 'rb'))

In [15]:
x_test = x_im_test

In [16]:
def get_model():
    input_ = Input(shape=(height, width, channels), name="X_1")
    
    # Layers 1+2
    out = Conv2D(9, (3, 3), padding='valid')(input_)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = Conv2D(18, (3, 3), padding='valid')(out)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = MaxPooling2D((2, 2), strides=(2, 2))(out)
    out = Dropout(0.1)(out)
    
    # Layers 2+3
    out = Conv2D(36, (3, 3), padding='valid')(out)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = Conv2D(72, (3, 3), padding='valid')(out)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = MaxPooling2D((2, 2), strides=(2, 2))(out)
#     out = Dropout(0.1)(out)

    # Layer 4
    out = Conv2D(144, (2, 2), padding='valid')(out)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = MaxPooling2D((2, 2), strides=(2, 2))(out)
#     out = Dropout(0.1)(out)
    
    # Layer 5
    out = Conv2D(256, (2, 2), padding='valid')(out)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = MaxPooling2D((2, 2), strides=(2, 2))(out)
#     out = Dropout(0.1)(out)
    
    out = Flatten()(out)
    
    # FCs 1+2+3
    out = Dense(1024, activation='relu')(out)
    out = Dropout(0.5)(out)
    out = Dense(196, activation='relu')(out)
    out = Dropout(0.1)(out)
    out = Dense(24, activation="softmax")(out)

    model = Model(input_, out)
    optimizer = Adam()
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    
    return model

model = get_model()

### KFold

In [18]:
kfold = 5
kfold_scores = []

results = []
sss = KFold(n_splits=kfold, random_state=0)
for i in range(5):
    
    model.load_weights('keras_models/kfold_w_arms_col/best_weights_'+str(i+1)+'.hdf5')

    # Predict
    preds_test = model.predict(x_test)    
    results.append(preds_test)
    
label_list = list(labels.columns)

In [19]:
label_list = list(labels.columns)
label_map_dict= {i.lower() : i for i in label_list}
combined_results = np.argmax(np.array(results).mean(axis=0), 1)
combined_results = [label_list[i] for i in combined_results]

In [20]:
b = set([i.lower() for i in labels.columns])
test['leakage_labs'] = test.Link_to_the_image.astype(str).map(lambda x :
                                                      list(set(x[x.rfind('/')+1:x.rfind('.')].lower() \
                                                      .split('-')).intersection(b)))
test['leakage_labs'] = test['leakage_labs'].map(lambda x : label_map_dict[x[0]] if len(x) > 0 else np.NaN)

In [21]:
test['Key'] = test.Link_to_the_image.astype(str).map(lambda x : x[x.rfind('/')+1:x.rfind('.')]+'.jpg')
test['prediction'] = test['Key'].map(lambda x : combined_results[link_test.index(x)] 
                                     if x in link_test else '')
test['Sub_category'] = test['leakage_labs'].fillna(test['prediction'])

In [22]:
test = test.drop(['Key', 'leakage_labs', 'prediction'], axis=1)

In [23]:
test = test.fillna('')
test.to_csv('./Submissions/Sub_offline.csv', index=False)

In [24]:
{x:combined_results.count(x) for x in combined_results}

{'Abstract': 251,
 'Biker': 4,
 'Camouflage': 27,
 'Checked': 21,
 'Colourblocked': 462,
 'Conversational': 151,
 'Floral': 289,
 'Geometric': 278,
 'Graphic': 1171,
 'Humour and Comic': 48,
 'People and Places': 125,
 'Self Design': 4,
 'Solid': 5814,
 'Sports': 1,
 'Sports and Team Jersey': 3,
 'Striped': 2337,
 'Superhero': 79,
 'Tribal': 1,
 'Typography': 3616,
 'Varsity': 19}